In [1]:
import xarray as xr
import dask
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import CloughTocher2DInterpolator, LinearNDInterpolator, NearestNDInterpolator
import scipy.spatial.qhull as qhull
import matplotlib.tri as mtri
import matplotlib.cm as cm
import os
from netCDF4 import Dataset, date2num
from datetime import datetime, timedelta
import pandas as pd
from pandas import Timestamp, to_datetime

#import dask.dataframe as dd
xr.set_options(display_style="text")
from function_02 import*

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(memory_limit='128GB')
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 400.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46580,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 400.00 GiB
Comm: tcp://127.0.0.1:33482,Total threads: 4
Dashboard: http://127.0.0.1:39339/status,Memory: 100.00 GiB
Nanny: tcp://127.0.0.1:34115,


In [2]:
data_address = '/work/ollie/nkolduno/output_orca12/ssh.fesom.1961.nc'
grid_address = '/work/ollie/nkolduno/meshes/FORCA12/fesom.mesh.diag.nc'
nod2dfile = '/work/ollie/nkolduno/meshes/FORCA12/nod2d.out'
elm2dfile = '/work/ollie/nkolduno/meshes/FORCA12/elem2d.out'
data = xr.open_dataset(data_address) #check if we can get effective solution if divided into seperate chunks
grid = xr.open_dataset(grid_address)
    
model_lon_nc = grid.lon.values
model_lat_nc = grid.lat.values


contentfile = pd.read_csv(
nod2dfile,
delim_whitespace=True,
skiprows=1,
names=["node_number", "x", "y", "flag"],
)
model_lon = contentfile.x.values
model_lat = contentfile.y.values


lon, lat = region_init()
lon2, lat2 = np.meshgrid(lon, lat)
lon2 = lon2.T
lat2 = lat2.T

Please input values in degree
Please enter with '-' sign if the values are in degree south or west
left: -70
right: 30
top: -60
bottom: -20


In [10]:
print(sum(model_lat - model_lat_nc))
print(sum(model_lon - model_lon_nc))


2.705701020831318e-10
-8.597204459035774e-09


In [11]:
file_content_elem = pd.read_csv(
    elm2dfile,
    delim_whitespace=True,
    skiprows=1,
    names=["first_elem", "second_elem", "third_elem"],
)
'''
file_content_elem['temp'] = file_content_elem['third_elem']
file_content_elem['third_elem'] = file_content_elem['second_elem']
file_content_elem['second_elem'] = file_content_elem['temp']
file_content_elem.drop('temp', inplace=True, axis = 1)
'''


"\nfile_content_elem['temp'] = file_content_elem['third_elem']\nfile_content_elem['third_elem'] = file_content_elem['second_elem']\nfile_content_elem['second_elem'] = file_content_elem['temp']\nfile_content_elem.drop('temp', inplace=True, axis = 1)\n"

In [12]:
elem = (file_content_elem.values - 1).astype('int32')
elem

array([[      0,       3,       6],
       [      0,       2,       1],
       [      0,       1,       3],
       ...,
       [8852360, 8852364, 8852362],
       [8852362, 8852364, 8852363],
       [8852363, 8852364, 8852365]], dtype=int32)

In [13]:
elem_nc = (grid.elements.data.astype('int32')-1).T
elem_nc

array([[      0,       6,       3],
       [      0,       1,       2],
       [      0,       3,       1],
       ...,
       [8852360, 8852362, 8852364],
       [8852362, 8852363, 8852364],
       [8852363, 8852365, 8852364]], dtype=int32)

In [7]:
%%time
triang, tri = triangulator(data, elem, model_lon, model_lat)

CPU times: user 3min 30s, sys: 11 s, total: 3min 41s
Wall time: 3min 41s


In [8]:
%%time
days = 1
#triangularized_data = [] 
NN_interpolator_list = []
points = np.vstack((model_lon, model_lat)).T
linear_interpolator_list = []
for day in range(days):
    data_sample = data.ssh[day,:].values
    #data_sample_scattered = client.scatter(data_sample)
    triangularized_lazy_data = data_triangulation(triang, tri, data_sample)(lon2, lat2)
    #NN_interpolator = nearest_nei_interpolator(triangularized_lazy_data, data_sample, model_lon, model_lat, lon2, lat2)
    nn_interpolation = NearestNDInterpolator(points, data_sample)((lon2, lat2))
    NN_interpolator_list.append(nn_interpolation)
    linear_interpolator_list.append(np.ma.masked_invalid(triangularized_lazy_data))

CPU times: user 4.52 s, sys: 544 ms, total: 5.06 s
Wall time: 5.22 s


In [14]:
linear_interpolator_list[0].shape

(1200, 480)

In [ ]:
for day in range(days):
    year = 1961
    out_path =  "/home/ollie/ssunar/output_files/"
    outfile = (out_path+'ssh_gridded_'+str(day)+'nn.nc')
    if os.path.isfile(outfile):
        os.remove(outfile)

    fw = Dataset(outfile, 'w')
    fw.createDimension('LONGITUDE', lon.size)
    fw.createDimension('LATITUDE', lat.size)

    lon3 = fw.createVariable('LONGITUDE', 'f', ('LONGITUDE',))
    lat3  = fw.createVariable('LATITUDE', 'f', ('LATITUDE',))
    ssh = fw.createVariable('ssh',np.float64, ('LONGITUDE', 'LATITUDE'))
    lat3.units = 'degrees_north'
    lat3.long_name = 'latitude'
    lon3.units = 'degrees_east'
    lon3.long_name = 'longitude'
    ssh.units = 'm'
    ssh.description = 'sea surface elevation'
    ssh.long_name = 'sea surface elevation'
    lon3[:] = lon[:]
    lat3[:] = lat[:]
    ssh[:] = NN_interpolator_list[day][:]
    fw.close()
    

In [ ]:
for day in range(days):
    year = 1961
    out_path =  "/home/ollie/ssunar/"
    outfile = (out_path+'ssh_gridded_'+str(day)+'linear.nc')
    if os.path.isfile(outfile):
        os.remove(outfile)

    fw = Dataset(outfile, 'w')
    fw.createDimension('LONGITUDE', lon.size)
    fw.createDimension('LATITUDE', lat.size)

    lon3 = fw.createVariable('LONGITUDE', 'f', ('LONGITUDE',))
    lat3  = fw.createVariable('LATITUDE', 'f', ('LATITUDE',))
    ssh = fw.createVariable('ssh',np.float64, ('LONGITUDE', 'LATITUDE'))
    lat3.units = 'degrees_north'
    lat3.long_name = 'latitude'
    lon3.units = 'degrees_east'
    lon3.long_name = 'longitude'
    ssh.units = 'm'
    ssh.description = 'sea surface elevation'
    ssh.long_name = 'sea surface elevation'
    lon3[:] = lon[:]
    lat3[:] = lat[:]
    ssh[day] = linear_interpolator_list[day][:]
    fw.close()

In [15]:
year = 1961
out_path =  "/home/ollie/ssunar/"
outfile = (out_path+'ssh_gridded_'+str(year)+'_'+str(0+1).zfill(3)+'_m.nc')
if os.path.isfile(outfile):
    os.remove(outfile)

In [16]:
year = 1961
out_path =  "/home/ollie/ssunar/"
outfile = (out_path+'ssh_gridded_'+str(year)+'_'+str(0+1).zfill(3)+'_m.nc')
if os.path.isfile(outfile):
    os.remove(outfile)

In [17]:
times = data.coords['time'].values
time_unit_out= "hours since 1950-01-01 00:00:00" #hours in case higher resolution output

In [18]:
fw = Dataset(outfile, 'w')

In [19]:
fw.createDimension('TIME', days)
fw.createDimension('LONGITUDE', lon.size)
fw.createDimension('LATITUDE', lat.size)

<class 'netCDF4._netCDF4.Dimension'>: name = 'LATITUDE', size = 480

In [20]:
lon3 = fw.createVariable('LONGITUDE', 'f', ('LONGITUDE',))
lat3  = fw.createVariable('LATITUDE', 'f', ('LATITUDE',))        
time = fw.createVariable('TIME', 'f', ('TIME',))                  
ssh = fw.createVariable('ssh',np.float64, ('TIME','LONGITUDE', 'LATITUDE'))


In [21]:
lat3.units = 'degrees_north'
lat3.long_name = 'latitude'
lon3.units = 'degrees_east'
lon3.long_name = 'longitude'

In [22]:
ssh.units = 'm'

In [23]:
ssh.description = 'sea surface elevation'
ssh.long_name = 'sea surface elevation'

In [24]:
time.setncattr('unit',time_unit_out)

In [25]:
print(time[:])

[--]


In [26]:
lon3[:] = lon[:]
print(lon3[:])

[-70.       -69.916664 -69.833336 ...  29.75      29.833334  29.916666]


In [27]:
lat3[:] = lat[:]

In [28]:
np.ma.array(lat3[:])

masked_array(data=[-60.      , -59.916668, -59.833332, -59.75    ,
                   -59.666668, -59.583332, -59.5     , -59.416668,
                   -59.333332, -59.25    , -59.166668, -59.083332,
                   -59.      , -58.916668, -58.833332, -58.75    ,
                   -58.666668, -58.583332, -58.5     , -58.416668,
                   -58.333332, -58.25    , -58.166668, -58.083332,
                   -58.      , -57.916668, -57.833332, -57.75    ,
                   -57.666668, -57.583332, -57.5     , -57.416668,
                   -57.333332, -57.25    , -57.166668, -57.083332,
                   -57.      , -56.916668, -56.833332, -56.75    ,
                   -56.666668, -56.583332, -56.5     , -56.416668,
                   -56.333332, -56.25    , -56.166668, -56.083332,
                   -56.      , -55.916668, -55.833332, -55.75    ,
                   -55.666668, -55.583332, -55.5     , -55.416668,
                   -55.333332, -55.25    , -55.166668, -55.083

In [29]:
from filters import bessel_high_filter

In [30]:
lat_ma = np.ma.array(lat3[:])
lon_ma = np.ma.array(lon3[:])
ssh_filtered = bessel_high_filter(linear_interpolator_list[0][:], wave_length = 500, y_c = lat_ma, x_c = lon_ma)

In [31]:
ssh_filtered

masked_array(
  data=[[0.3261029742076711, 0.3404197357993972, 0.358913947682036, ...,
         --, --, --],
        [0.3078938103229891, 0.31714083758272427, 0.33140274588770624,
         ..., --, --, --],
        [0.2915236482246899, 0.2955035521873963, 0.30441971637791077,
         ..., --, --, --],
        ...,
        [-0.011140546820143093, -0.004371494440427837,
         0.0017834607584696904, ..., --, --, --],
        [-0.02604310869818338, -0.018913827196874955,
         -0.01303552835243904, ..., --, --, --],
        [-0.04050973830522664, -0.03311461979246899,
         -0.027233241758049198, ..., --, --, --]],
  mask=[[False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        ...,
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True]],
  fill_value=1e

In [32]:
linear_interpolator_list[0][:]

masked_array(
  data=[[-0.7143123778390761, -0.6984908574360169, -0.6780314868011601,
         ..., --, --, --],
        [-0.736072734466255, -0.7252736631432999, -0.7090037464076833,
         ..., --, --, --],
        [-0.7559842912065307, -0.7504108185442426, -0.7394504389535257,
         ..., --, --, --],
        ...,
        [-1.6210185401138304, -1.6149425907917099, -1.6094164332593666,
         ..., --, --, --],
        [-1.6354449879248314, -1.6290634685550387, -1.623864878344781,
         ..., --, --, --],
        [-1.649481354220626, -1.6428911401034725, -1.6377428320789145,
         ..., --, --, --]],
  mask=[[False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        ...,
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True]],
  fill_value=1e+20)

In [35]:
for day in range(days): 
    ts=Timestamp(times[day])
    t=ts.to_pydatetime()
    ssh[day] = linear_interpolator_list[day][:]
    time[day] = date2num(t, time_unit_out)
    


In [38]:
fw

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): TIME(2), LONGITUDE(2400), LATITUDE(960)
    variables(dimensions): float32 LONGITUDE(LONGITUDE), float32 LATITUDE(LATITUDE), float32 TIME(TIME), float64 ssh(TIME, LONGITUDE, LATITUDE)
    groups: 

In [127]:
fw.close()

In [2]:
f = Dataset("/work/ollie/bpanthi/ssh_gridded_1961_0011_m.nc", 'r')


In [3]:
f


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): TIME(30), LONGITUDE(2400), LATITUDE(960)
    variables(dimensions): float32 LONGITUDE(LONGITUDE), float32 LATITUDE(LATITUDE), float32 TIME(TIME), float64 ssh(TIME, LONGITUDE, LATITUDE)
    groups: 

In [24]:
for i in range(30):
    print(f['ssh'][i])
    print("j")



[[-- -- -- ... -0.6984711603885305 -0.7070102198134354
  -0.7143123778390752]
 [-- -- -- ... -0.7117072098175967 -0.7191209154591327 -0.725532223320398]
 [-- -- -- ... -0.7252698746766981 -0.731325677350048 -0.7360815255514463]
 ...
 [-- -- -- ... -1.642880114790831 -1.6458187897243501 -1.649475771878043]
 [-- -- -- ... -1.6493299862069382 -1.6524329957232533
  -1.6561745840720317]
 [-- -- -- ... -1.6557334842715719 -1.6589661225492787 -1.662809899413439]]
j
[[-- -- -- ... -0.7097579344823863 -0.7146095689448817
  -0.7183948253424273]
 [-- -- -- ... -0.7209127106385331 -0.7245620725293556
  -0.7274268055368207]
 [-- -- -- ... -0.7324203005169423 -0.7344822941670532
  -0.7354428197540486]
 ...
 [-- -- -- ... -1.647576861785236 -1.6511817458815017 -1.6554676964731314]
 [-- -- -- ... -1.6552091128863857 -1.6587511551229461
  -1.6630270705029986]
 [-- -- -- ... -1.662874614187916 -1.6663547115460347 -1.6706089166428448]]
j
[[-- -- -- ... -0.7048020304998186 -0.7065011013380555
  -0.7074100

[[-- -- -- ... -0.8948357632721482 -0.8936534464600925
  -0.8929170660760839]
 [-- -- -- ... -0.9052767726708346 -0.9042252976920615
  -0.9035772739586712]
 [-- -- -- ... -0.9158720519693411 -0.9152785512961543
  -0.9151954268432831]
 ...
 [-- -- -- ... -1.5388566091955038 -1.5359243613646907
  -1.5339830562356926]
 [-- -- -- ... -1.5264084558200501 -1.5236888944648346
  -1.5221175697619387]
 [-- -- -- ... -1.5136981812055126 -1.5110896998624668
  -1.5098399852348834]]
j
[[-- -- -- ... -0.8893565382660142 -0.8875766136246774
  -0.8859138767982913]
 [-- -- -- ... -0.8993084590244678 -0.8976590093420604
  -0.8960260668410491]
 [-- -- -- ... -0.9092636141702748 -0.9078055855194762 -0.906469355899219]
 ...
 [-- -- -- ... -1.520244337646318 -1.5166132943202797 -1.5141822077321923]
 [-- -- -- ... -1.5069881016511069 -1.5033307037534875
  -1.5010091275353385]
 [-- -- -- ... -1.4936862880269786 -1.4899386639737688
  -1.4876734290734408]]
j
[[-- -- -- ... -0.9001134082689788 -0.8978509046614782